In [3]:
import numpy as np
from PIL import Image
import os
import cv2
import xml.etree.ElementTree as ET
#import skimage
from skimage import morphology

In [ ]:
Image.open("/data/unagi0/kanayama/dataset/nuclei_images/Tissue_images/TCGA-18-5592-01Z-00-DX1.tif-1.png")

In [ ]:
tree = ET.parse('/data/unagi0/kanayama/dataset/nuclei_images/Tissue_annotations/TCGA-KB-A93J-01A-01-TS1.xml')
elem = tree.getroot()

In [ ]:
for child in root:
    print(child.tag, child.attrib)

In [ ]:
elem.tag

In [ ]:
root.get()

In [ ]:
elem.keys()

In [ ]:
elem.items()

In [ ]:
# すべての要素にアクセス
for e in elem.getiterator():
    print(e.tag)

In [ ]:
# 子要素のリストを取得(再帰的ではなく、直系の子要素のみ)
for e in list(elem):
    print(e.tag)

In [ ]:
elem[0].keys()

In [ ]:
elem[0][1][1][1][0].get("X")

In [ ]:
filename= "TCGA-18-5592-01Z-00-DX1"
image_filename = "/data/unagi0/kanayama/dataset/nuclei_images/Tissue_images/" + filename + ".tif-0.png"
annotation_filename = "/data/unagi0/kanayama/dataset/nuclei_images/Tissue_annotations/" + filename + ".xml"

In [ ]:
img = Image.open(image_filename)
tree = ET.parse(annotation_filename)
elem = tree.getroot()

In [6]:
# mask画像の生成
path = "/data/unagi0/kanayama/dataset/nuclei_images/"

omit_list = ["TCGA-CH-5767-01Z-00-DX1",
                     "TCGA-E2-A1B5-01Z-00-DX1", 
                     "TCGA-G9-6336-01Z-00-DX1",
                     "TCGA-G9-6348-01Z-00-DX1",
                     "TCGA-G9-6363-01Z-00-DX1",
                     "TCGA-HE-7128-01Z-00-DX1",
                     "TCGA-HE-7129-01Z-00-DX1",
                     "TCGA-HE-7130-01Z-00-DX1"]

for fn in os.listdir(path + "Tissue_annotations/"):
    filename = fn.split(".")[0]
    if filename in omit_list:
        print("omit!!!")
        continue
        
    print("processing ", filename)
    image_filename = path + "Tissue_images/" + filename +  ".tif-0.png"
    image_filename2 = path + "Tissue_images/" + filename +  ".tiff.png"
    annotation_filename = path + "Tissue_annotations/" + filename +  ".xml"
    save_mask_filename = path + "stage1_train_default/masks/" + image_filename.split("/")[-1]
    save_edge_filename = path + "stage1_train_default/edges/" + image_filename.split("/")[-1]
    try:
        img = Image.open(image_filename)
    except FileNotFoundError:
        img = Image.open(image_filename2)
        #continue
        
    tree = ET.parse(annotation_filename)
    elem = tree.getroot()

    mask = np.zeros(img.size).astype(np.uint8)
    edge = np.zeros(img.size).astype(np.uint8)
    
    #print(len(elem[0][1]))
    for nuclei in elem[0][1]:
        edge_one = np.ones(img.size).astype(np.uint8) * 255
        for one_dot in nuclei[1]:
            x = int(float(one_dot.get("X"))) - 1
            y = int(float(one_dot.get("Y"))) - 1
            try:
                edge_one[y][x] = 0
            except IndexError:
                pass
            
        edge_one_erode = cv2.erode(edge_one, np.ones((5, 5)), iterations = 1)
        mask_one, num = morphology.label(edge_one_erode, neighbors=4, return_num=True)
        mask = np.logical_or(mask, (mask_one >= 2))
        mask = np.logical_and(mask, edge_one)
        edge = np.logical_or(edge, 255 - edge_one)
        edge = np.logical_and(edge, (mask_one < 2))
        
    mask = Image.fromarray(mask.astype(np.uint8) * 255)
    mask.save(save_mask_filename)
    #edge = Image.fromarray(edge.astype(np.uint8) * 255)
    edge = Image.fromarray(np.zeros(edge.shape).astype(np.uint8)) #edgeは真っ黒
    edge.save(save_edge_filename)

processing  TCGA-18-5592-01Z-00-DX1
481
processing  TCGA-21-5784-01Z-00-DX1
399
processing  TCGA-21-5786-01Z-00-DX1
441
processing  TCGA-38-6178-01Z-00-DX1
425
processing  TCGA-49-4488-01Z-00-DX1
558
processing  TCGA-50-5931-01Z-00-DX1
446
processing  TCGA-A7-A13E-01Z-00-DX1
315
processing  TCGA-A7-A13F-01Z-00-DX1
357
processing  TCGA-AR-A1AK-01Z-00-DX1
434
processing  TCGA-AR-A1AS-01Z-00-DX1
406
processing  TCGA-AY-A8YK-01A-01-TS1
364
processing  TCGA-B0-5698-01Z-00-DX1
358
processing  TCGA-B0-5710-01Z-00-DX1
360
processing  TCGA-B0-5711-01Z-00-DX1
343
omit!!!
processing  TCGA-DK-A2I6-01A-01-TS1
343
processing  TCGA-E2-A14V-01Z-00-DX1
379
omit!!!
processing  TCGA-G2-A2EK-01A-02-TSB
402
omit!!!
omit!!!
processing  TCGA-G9-6356-01Z-00-DX1
443
processing  TCGA-G9-6362-01Z-00-DX1
473
omit!!!
omit!!!
omit!!!
omit!!!
processing  TCGA-KB-A93J-01A-01-TS1
1392
processing  TCGA-NH-A8F7-01A-01-TS1
364
processing  TCGA-RD-A8N9-01A-01-TS1
1166


In [ ]:
Image.fromarray(mask.astype(np.uint8) * 255)

In [ ]:
Image.fromarray(mask.astype(np.uint8) * 255)

In [ ]:
mask_one = np.ones(img.size).astype(np.uint8) * 255
for one_dot in elem[0][1][4][1]:
        x = int(float(one_dot.get("X"))) - 1
        y = int(float(one_dot.get("Y"))) - 1
        mask_one[y][x] = 0
mask_one = morphology.label(mask_one, neighbors=4, background=1)

In [ ]:
np.unique(mask_one, return_index=True)

In [ ]:
Image.fromarray((mask_one == 1).astype(np.uint8) * 255)

In [ ]:
Image.open("/data/unagi0/kanayama/dataset/nuclei_images/Tissue_annotations/TCGA-18-5592-01Z-00-DX1.png")

In [ ]:
filename= "TCGA-18-5592-01Z-00-DX1"
image_filename = "/data/unagi0/kanayama/dataset/nuclei_images/Tissue_images/" + filename + ".tif-0.png"
annotation_filename = "/data/unagi0/kanayama/dataset/nuclei_images/Tissue_annotations/" + filename + ".xml"
Image.open( "/data/unagi0/kanayama/dataset/nuclei_images/Tissue_annotations/" + filename + ".png")